In [ ]:
#импорттау
import pygame, sys
from pygame.locals import *
import random, time

#пайгеймды бастау
pygame.init()

FPS = 60 #кадр жиілігі (кадр/сек)
FramePerSec = pygame.time.Clock() #сағатты орнатамыз

#түстерді RGB форматында белгілейміз
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

#терезенің өлшемдері
width = 400
height = 600
SPEED = 3  #бастапқы жылдамдық
SCORE = 0  #ұпай
COINS = 0  #жиналған монеталар саны

#шрифттерді орнату
font = pygame.font.SysFont("Verdana", 20)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)  # "Game Over" жазуы

#фондық суретті жүктейміз
background = pygame.image.load(r"C:\Users\nurga\OneDrive\Desktop\Projects\PP2\labs1\lab_8\images\AnimatedStreet.png")

#ойын терезесін құрамыз
screen = pygame.display.set_mode((400, 600))
screen.fill(WHITE)
pygame.display.set_caption("Racer")  #ойын атауын қоямыз

#монета жинаған сайын жылдамдық артуын қадағалайтын айнымалылар
c1, c2, c3, c4, c5 = False, False, False, False, False

#дұшпан класы (жолдағы көлік)
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load(r"C:\Users\nurga\OneDrive\Desktop\Projects\PP2\labs1\lab_8\images\Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, width - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if (self.rect.top > 600):
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, width - 40), 0)
#монета класы
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load(r"C:\Users\nurga\OneDrive\Desktop\Projects\PP2\labs1\lab_8\images\Coin.png")
        self.image = pygame.transform.scale(self.image, (40, 40))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, width - 40), random.randint(40, height - 40))
        self.value = random.randint(1, 3)  #кездейсоқ тиын

    def move(self):
        global COINS, SPEED, c1, c2, c3, c4, c5
        COINS += self.value 
        self.value = random.randint(1, 3) 
        self.rect.top = random.randint(40, width - 40)
        self.rect.center = (random.randint(40, width - 40), random.randint(40, height - 40))

        if not c1 and COINS >= 10:
            SPEED += 1
            c1 = True
        if not c2 and COINS >= 20:
            SPEED += 1
            c2 = True
        if not c3 and COINS >= 30:
            SPEED += 1
            c3 = True
        if not c4 and COINS >= 40:
            SPEED += 1
            c4 = True
        if not c5 and COINS >= 50:
            SPEED += 1
            c5 = True

#ойыншы класы
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load(r"C:\Users\nurga\OneDrive\Desktop\Projects\PP2\labs1\lab_8\images\Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < width and pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)
        if self.rect.top > 0 and pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if self.rect.bottom < height and pressed_keys[K_DOWN]:
            self.rect.move_ip(0, 5)

#ойын аяқталған кезде шығатын экран
def game_over_screen():
    screen.fill(RED)
    screen.blit(game_over, (30, 250))
    pygame.display.update()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_SPACE:
                    return True
                elif event.key == K_ESCAPE:
                    return False 

#ойынды бастау функциясы
def init_game():
    global P1, E1, C1, enemies, coinss, all_sprites, INC_SPEED
    global SPEED, SCORE, COINS, c1, c2, c3, c4, c5

    SPEED = 3
    SCORE = 0
    COINS = 0
    c1, c2, c3, c4, c5 = False, False, False, False, False

    P1 = Player()
    E1 = Enemy()
    C1 = Coin()

    enemies = pygame.sprite.Group()
    enemies.add(E1)
    coinss = pygame.sprite.Group()
    coinss.add(C1)
    all_sprites = pygame.sprite.Group()
    all_sprites.add(P1)
    all_sprites.add(E1)
    all_sprites.add(C1)

    INC_SPEED = pygame.USEREVENT + 1
    pygame.time.set_timer(INC_SPEED, 1000)

init_game()
background_y = 0
#негізгі код
while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.1
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    if pygame.sprite.spritecollideany(P1, enemies):
        if game_over_screen():
            init_game()
            continue
        else:
            pygame.quit()
            sys.exit()

    background_y = (background_y + SPEED) % background.get_height()
    screen.blit(background, (0, background_y))
    screen.blit(background, (0, background_y - background.get_height()))

    scores = font_small.render(str(SCORE), True, BLACK)
    screen.blit(scores, (10, 10))

    coins = font_small.render(str(COINS), True, BLACK)
    screen.blit(coins, (370, 10))

    for entity in all_sprites:
        screen.blit(entity.image, entity.rect)
        if entity == C1:
            if pygame.sprite.spritecollideany(P1, coinss):
                entity.move()
        else:
            entity.move()

    for enemy in enemies:
        enemy.move()

    for coin in coinss:
        coin.rect.y += SPEED
        if coin.rect.top > height:
            coin.rect.y = -coin.rect.height
            coin.rect.x = random.randint(40, width - 40)

    pygame.display.update()
    FramePerSec.tick(FPS)